In [ ]:
import importlib
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from os.path import join
import os
import seaborn as sns
from torchvision.ops.misc import interpolate
from tqdm.notebook import tqdm

#### Custum libraries
import lib.algos_maxRSA as max_rsa
import lib.utils_RSA as rsa
import lib.utils_CKA as cka
from lib.algos import *


importlib.reload(rsa)
importlib.reload(cka)
importlib.reload(max_rsa)

In [ ]:
dataset = 'ecoLennyTest'
models  = ['ego', 'saycam', 'imagenet', 'supervised', 'resnet']
#models  = ['ego', 'saycam']
path2activations = f'/home/alban/Documents/activations_datadriven/%s_{dataset}/'

imagelists = {}
activations = {}
for model in models:
    with open(join(path2activations%model, 'imagepaths.txt'), 'r') as f:
        imagelists[model] = [line.strip() for line in f.readlines()]
    activations[model] = np.load(join(path2activations % model, 'cls_tokens.npy'))

imagelist = imagelists[model]
activations[model].shape

In [ ]:
#### Normalize vectors
for model in models:
    norms = np.linalg.norm(activations[model], axis=1, keepdims=True)
    activations[model] = activations[model]/norms # normalization

In [ ]:
### check if images were shown in the same order
assert imagelists[models[0]] == imagelists[models[1]]
imagelist = imagelists[models[0]] # since they are the same, only consider one list

#### check if each category has the same number of images and list all categories in listcats
count = 0
cat = ''
listcat = list()
for i, imgp in enumerate(imagelist):
    current_cat = imgp.split('/')[-2]
    if i == 0:
        cat = current_cat
        listcat.append(current_cat)
    if cat != current_cat:
        cat = current_cat
        listcat.append(current_cat)
        count = 1
    else:
        count += 1

nb_per_cat = count # in val, 50 images per cate

nb_per_cat

In [ ]:
### reshape activations according to include categories
cat_activations = activations.copy()

for model in models:
    shape = activations[model].shape
    cat_activations[model] = activations[model].reshape(-1, nb_per_cat, shape[-1])

In [ ]:
compactness, compact_categories = max_rsa.compute_compactness(cat_activations, models, listcat, measure = 'Fisher_discriminant')

In [ ]:
max_rsa.plot_stats_one(compactness,models,  ['Categories', 'Normalized var'])

In [ ]:
def sample_catrdm_pairs(cat_activations, submodels, n_samples=1000, nb_subcategories=12, nb_per_category = 50,
                                    batch_size=10, seed=None):
    """
    Memory-efficient version that processes in batches and optionally saves to disk.

    Parameters:
    -----------
    batch_size : int
        Number of samples to process at once (default: 1000)
    output_file : str, optional
        If provided, saves results to this file using pickle
    """

    if seed is not None:
        np.random.seed(seed)

    dissimilarity_metric = 'L2squared'

    nb_categories = len(cat_activations[submodels[0]])
    n_batches = (n_samples + batch_size - 1) // batch_size

    all_sims_samples = []
    all_indices = []
    print(f"Processing {n_samples} samples in {n_batches} batches of {batch_size}...")

    batch_rdms = {}
    for batch_idx in tqdm(range(n_batches)):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, n_samples)
        current_batch_size = end_idx - start_idx

        subset_size = nb_subcategories
        # Allocate batch arrays
        batch_sim = np.zeros((current_batch_size))
        batch_indices = np.zeros((current_batch_size, subset_size), dtype=int)
        for model in submodels:
            batch_rdms[model] = np.zeros((current_batch_size, nb_subcategories*nb_per_category, nb_subcategories*nb_per_category))
        for i in range(current_batch_size):
            # Randomly select images
            cat_indices = np.random.choice(nb_categories, size=nb_subcategories, replace=False)

            # Compute subrdms
            for model in submodels:
                batch_rdms[model][i] = rsa.compute_RDMs(cat_activations[model][cat_indices].reshape(nb_subcategories*nb_per_category, -1),
                            metric=dissimilarity_metric, display=False)
            # Extract submatrices
            batch_sim[i] = rsa.Compute_sim_RDMs(batch_rdms[submodels[0]][i], batch_rdms[submodels[1]][i], center = False, metric = 'pearson' )
            batch_indices[i] = cat_indices

        all_sims_samples.append(batch_sim)
        all_indices.append(batch_indices)

    # Concatenate all batches
    sim_samples = np.concatenate(all_sims_samples, axis=0)
    indices_used = np.concatenate(all_indices, axis=0)


    return sim_samples, indices_used




In [ ]:
for i, model1 in enumerate(models[:-1]):
    for j, model2 in enumerate(models[i+1:]):
        sim_samples, indices_used = sample_catrdm_pairs(cat_activations, [model1, model2], n_samples=500, nb_subcategories = 12, nb_per_category = 50, batch_size=10, seed=None)
        np.save(f'results/categories_sim_samples_{model1}_{model2}_{dataset}.npy', sim_samples)

We have an idea of the average pearson similarity found if we select 12 categories - pretty high!
Can we find a subset of 12 categories that has a much lover similarity than that. For example, categories with the lowest correlations between the models.

In [ ]:
def subsimilar_categories(cat_activations, submodels, dissimilarity_metric = 'L2squared', similarity_metric = 'pearson', nb_subcategories = 12):
    assert len(submodels)== 2
    assert cat_activations[submodels[0]].shape[:2] == cat_activations[submodels[1]].shape[:2]

    shape = cat_activations[submodels[0]].shape

    nb_categories = shape[0]
    nb_per_categories = shape[1]

    mean_cat_activations1 = cat_activations[submodels[0]].mean(axis = 1)
    mean_cat_activations2 = cat_activations[submodels[1]].mean(axis = 1)

    RDM1 = rsa.compute_RDMs(mean_cat_activations1,
                            metric=dissimilarity_metric, display=False)
    RDM2 = rsa.compute_RDMs(mean_cat_activations2,
                            metric=dissimilarity_metric, display=False)

    RDM1_centered = RDM1 - np.mean(RDM1)
    RDM2_centered = RDM2 - np.mean(RDM2)

    #RDM1_centered = RDM1_centered / np.sqrt(np.sum(RDM1_centered ** 2))
    #RDM2_centered = RDM2_centered / np.sqrt(np.sum(RDM2_centered ** 2))

    correlations = np.sum(RDM1_centered * RDM2_centered, axis=0)
    subsimiliar_categories = np.argsort(correlations)[:nb_subcategories]

    '''#subsimilar_RDM1 = rsa.compute_RDMs(cat_activations[submodels[0]][subsimiliar_categories].reshape(nb_subcategories*nb_per_categories, -1),
                            metric=dissimilarity_metric, display=False)
    #subsimilar_RDM2 = rsa.compute_RDMs(cat_activations[submodels[1]][subsimiliar_categories].reshape(nb_subcategories*nb_per_categories, -1),
                            metric=dissimilarity_metric, display=False)
    #print(rsa.Compute_sim_RDMs(subsimilar_RDM1, subsimilar_RDM2, metric = similarity_metric))'''
    return correlations, subsimiliar_categories




In [ ]:
cat_similarities = {}
similarities = {}
for i, model1 in enumerate(models[:-1]):
    for j, model2 in enumerate(models[i+1:]):
        correlations, subsimilar_cats = subsimilar_categories(cat_activations, [model1, model2])
        RDM1, RDM2, RDM1_sorted, RDM2_sorted, sorted_indices = max_rsa.find_subsimilar_subset(cat_activations, [model1, model2], subsimilar_cats,  images_per_subset = 4, nb_per_category = 50)
        cat_sim = rsa.Compute_sim_RDMs(RDM1, RDM2, metric = 'pearson')
        sim = rsa.Compute_sim_RDMs(RDM1_sorted, RDM2_sorted, metric = 'pearson')

        '''fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
        subs[0].imshow(RDM1, cmap='gray')
        subs[1].imshow(RDM2, cmap='gray')
        subs[0].axis('off')
        subs[1].axis('off')
        fig.suptitle(f'{cat_sim}')
        fig.tight_layout()'''
        cat_similarities[f'{model1}_{model2}'] = cat_sim
        print(subsimilar_cats)

        fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
        subs[0].imshow(RDM1_sorted, cmap='gray')
        subs[1].imshow(RDM2_sorted, cmap='gray')
        subs[0].axis('off')
        subs[1].axis('off')
        fig.suptitle(f'{sim}')
        fig.tight_layout()
        plt.show()
        plt.close()
        similarities[f'{model1}_{model2}'] = sim

In [ ]:
cat_similarities_compact = {}
similarities_compact = {}
sorted_indices = {}
for i, model1 in enumerate(models[:-1]):
    for j, model2 in enumerate(models[i+1:]):
        nb_categories = len(listcat)
        labels, sortedmaxdiffcats, maxdiffs = max_rsa.max_compactness_difference(
                compact_categories, compactness, nb_categories, listcat, models = [model1, model2],
                nb_considered_categories = 12, compactness_diff_measure = 'normalizedDiff'
            )
        RDM1, RDM2, RDM1_sorted, RDM2_sorted, sorted_indices[f'{model1}_{model2}'] = max_rsa.find_subsimilar_subset(cat_activations, [model1, model2], labels[:12],  images_per_subset = 4, nb_per_category = 50)
        cat_sim = rsa.Compute_sim_RDMs(RDM1, RDM2, metric = 'pearson')
        sim = rsa.Compute_sim_RDMs(RDM1_sorted, RDM2_sorted, metric = 'pearson')
        '''fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
        subs[0].imshow(RDM1, cmap='gray')
        subs[1].imshow(RDM2, cmap='gray')
        subs[0].axis('off')
        subs[1].axis('off')
        fig.suptitle(f'{rsa.Compute_sim_RDMs(RDM1, RDM2, metric = 'pearson')}')
        fig.tight_layout()'''
        cat_similarities_compact[f'{model1}_{model2}'] = cat_sim

        fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
        subs[0].imshow(RDM1_sorted, cmap='gray')
        subs[1].imshow(RDM2_sorted, cmap='gray')
        subs[0].axis('off')
        subs[1].axis('off')
        fig.suptitle(f'{rsa.Compute_sim_RDMs(RDM1_sorted, RDM2_sorted, metric = 'pearson')}')
        fig.tight_layout()
        similarities_compact[f'{model1}_{model2}'] = sim

In [ ]:
dataset

In [ ]:
import glob
listsamples = glob.glob(f'results/categories_sim_samples*{dataset}.npy')
nb_cols = 5
fig, subs = plt.subplots(nrows = 2, ncols = nb_cols, figsize = (25,10), sharex = True, sharey = True)
for f, file in enumerate(listsamples):
    sample = np.load(file)
    hist, bin_edges = np.histogram(sample, 100)
    subs[int(f/nb_cols), f%nb_cols].bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], linewidth = 0, align = 'edge')
    #subs[f//5, f%5].legend()
    subs[int(f/nb_cols), f%nb_cols].set_xlabel('Similarity')
    subs[int(f/nb_cols), f%nb_cols].set_ylabel('Density')
    subs[int(f/nb_cols), f%nb_cols].set_title(f'{file.split('_')[-3]}_{file.split('_')[-2]}')
    name = f'{file.split('_')[-3]}_{file.split('_')[-2]}'
    subs[int(f/nb_cols), f%nb_cols].vlines(cat_similarities[name],0,1, 'g')
    subs[int(f/nb_cols), f%nb_cols].vlines(cat_similarities_compact[name],0,1, 'r')


plt.tight_layout()
plt.show()
fig.savefig(f'figures/compactness/categories_selectionVSdistribution_{dataset}.png')
plt.close()

In [ ]:
import pickle
listpickles_ecoLennyTest = glob.glob(f'/home/alban/Documents/results_image_selection/{dataset}_*.pkl')

RESULTS = {}
for p, pkl in enumerate(listpickles_ecoLennyTest):
    name = pkl.split('/')[-1][:-4]
    f = open(pkl, "rb")
    RESULTS[name] = pickle.load(f)
    f.close()

list_names = [k for k in RESULTS.keys()]
similarities_compactness = {}

similarities_compactness = [RESULTS[f'{dataset}_Truenormalize_silhouette_score_normalizedDiff_pearson']['similarity_dict'][pair]['similarity'] for pair in
                      RESULTS[f'{dataset}_Truenormalize_silhouette_score_normalizedDiff_pearson']['similarity_dict'].keys()]

In [ ]:
import glob

nb_cols = 5
fig, subs = plt.subplots(nrows = 2, ncols = nb_cols, figsize = (25,10), sharex = True, sharey = True)
f = 0
for i, model1 in enumerate(models[:-1]):
    for j, model2 in enumerate(models[i+1:]):
        sample = np.load(f'results/sim_samples_{model1}_{model2}_{dataset}.npy')
        hist, bin_edges = np.histogram(sample, 100)
        subs[int(f/nb_cols), f%nb_cols].bar(bin_edges[:-1],hist/max(hist), width = bin_edges[1] - bin_edges[0], linewidth = 0, align = 'edge')
        #subs[f//5, f%5].legend()
        subs[int(f/nb_cols), f%nb_cols].set_xlabel('Similarity')
        subs[int(f/nb_cols), f%nb_cols].set_ylabel('Density')
        subs[int(f/nb_cols), f%nb_cols].set_title(f'{model1}_{model2}')
        subs[int(f/nb_cols), f%nb_cols].vlines(similarities[f'{model1}_{model2}'],0,1, 'green')
        subs[int(f/nb_cols), f%nb_cols].vlines(similarities_compactness[f],0,1, 'r')
        subs[int(f/nb_cols), f%nb_cols].vlines(similarities_compact[f'{model1}_{model2}'],0,1, 'y')
        #subs[int(f/nb_cols), f%nb_cols].vlines(sample.mean(),0,1, 'k')
        subs[int(f/nb_cols), f%nb_cols].set_xlim(-0.21,0.9)
        f +=1


plt.tight_layout()
plt.show()
fig.savefig(f'figures/compactness/selectionVSdistribution_{dataset}.png')
plt.close()

In [ ]:
### Comparing the RDMs for the 2 dataset
random_categories = [50,89,260,478,12,205,401,256,369,78]
submodels = ['saycam', 'imagenet']
for dataset in ['ecoVal', 'ecoLennyTest']:
    models  = ['ego', 'saycam', 'imagenet', 'supervised', 'resnet']
    #models  = ['ego', 'saycam']
    path2activations = f'/home/alban/Documents/activations_datadriven/%s_{dataset}/'

    imagelists = {}
    activations = {}
    for model in models:
        with open(join(path2activations%model, 'imagepaths.txt'), 'r') as f:
            imagelists[model] = [line.strip() for line in f.readlines()]
        activations[model] = np.load(join(path2activations % model, 'cls_tokens.npy'))

    activations[model].shape
    #### Normalize vectors
    for model in models:
        norms = np.linalg.norm(activations[model], axis=1, keepdims=True)
        activations[model] = activations[model]/norms # normalization

    ### reshape activations according to include categories
    cat_activations = activations.copy()
    for model in models:
        shape = activations[model].shape
        cat_activations[model] = activations[model].reshape(-1, nb_per_cat, shape[-1])

    shape = activations[model].shape
    cat_activations[model] = activations[model].reshape(-1, nb_per_cat, shape[-1])

    cat_activations_subset1 = cat_activations[submodels[0]][random_categories]
    cat_activations_subset2 = cat_activations[submodels[1]][random_categories]

    cat_shape = cat_activations_subset1.shape

    RDM1 = rsa.compute_RDMs(cat_activations_subset1.reshape(cat_shape[0] * cat_shape[1], -1),
                            metric='L2squared', display=False)
    RDM2 = rsa.compute_RDMs(cat_activations_subset2.reshape(cat_shape[0] * cat_shape[1], -1),
                            metric='L2squared', display=False)

    fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
    subs[0].imshow(RDM1, cmap='gray')
    subs[1].imshow(RDM2, cmap='gray')
    subs[0].axis('off')
    subs[1].axis('off')
    fig.suptitle(f'{rsa.Compute_sim_RDMs(RDM1, RDM2, metric = 'pearson')}')
    fig.tight_layout()

In [ ]:
## Looking at selections
#imagelist = [img.replace('/raid/leonard_vandyck/datasets/genloc/', '/home/alban/Documents/ecoLennyTest/') for img in imagelist]
imagelist = [img.replace('/raid/shared/datasets/visoin/ecoset/', '/home/alban/Documents/ecoset/') for img in imagelist]
imagespaths = {}
for i, model1 in enumerate(models[:-1]):
    for j, model2 in enumerate(models[i+1:]):
        savename = f'Truenormalize_Fisher_discriminant_corr_{model1}_{model2}_ecoTest'
        images, imagespaths[model1 + '_' + model2] = max_rsa.display_low_similarity_images(imagelist, sorted_indices[f'{model1}_{model2}'], n_images=48,
                                                      grid_cols=8, figsize=(20, 12),
                                                      save_path=f'figures/compactness/subset/{savename}.png')

In [ ]:
imagelist

In [ ]:
RDMs = {}
metric = 'L2squared'
for i, model in enumerate(models):
    print(model)
    RDMs[model] = rsa.compute_RDMs(activations[model], metric = metric, display = False, title = f'{model}_{metric}')

In [ ]:
mean_RDM = RDMs.copy()
for model in models:
    mean_RDM[model] = RDMs[model].reshape(len(listcat), nb_per_cat, len(listcat), nb_per_cat)
    mean_RDM[model] = mean_RDM[model].transpose(0, 2, 1, 3)
    mean_RDM[model] = mean_RDM[model].mean(axis = (2,3))

fig, subs = plt.subplots(1,2, sharex=True, sharey=True)
subs[0].imshow(mean_RDM[submodels[0]], cmap='gray')
subs[1].imshow(mean_RDM[submodels[1]], cmap='gray')
subs[0].axis('off')
subs[1].axis('off')
fig.suptitle(f'{rsa.Compute_sim_RDMs(mean_RDM[submodels[0]], mean_RDM[submodels[1]], metric = 'pearson')}')
fig.tight_layout()

In [ ]:
submodels = ['saycam', 'ego']
nb_categories = len(listcat)
labels, sortedmaxdiffcats, maxdiffs = max_rsa.max_compactness_difference(
                compact_categories, compactness, nb_categories, listcat, models = submodels,
                nb_considered_categories = 12, compactness_diff_measure = 'normalizedDiff'
            )

In [ ]:
from itertools import combinations
categories = labels[:12]
dissimilarity_metric = 'L2squared'
images_per_subset = 4

#### First build the RDMs using all images of the chosen categories to get the general stats
cat_activations_subset1 = cat_activations[submodels[0]][categories]
cat_activations_subset2 = cat_activations[submodels[1]][categories]

cat_shape = cat_activations_subset1.shape

RDM1 = rsa.compute_RDMs(cat_activations_subset1.reshape(cat_shape[0]*cat_shape[1], -1), metric = dissimilarity_metric, display = False)
RDM2 = rsa.compute_RDMs(cat_activations_subset2.reshape(cat_shape[0] * cat_shape[1], -1),
                        metric=dissimilarity_metric, display=False)
means = {}
n = len(RDM1)
upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
means['x'] = np.mean(RDM1[upper_indices])
means['y'] = np.mean(RDM2[upper_indices])
means['norm'] = np.std(RDM1[upper_indices]) * np.std(RDM2[upper_indices])
print(means)
for c, category in enumerate(tqdm(categories[:2], desc="Processing categories")):
    print(f"\nProcessing category: {category}")
    # Get activations for both models for this category
    cat_RDM1 = RDM1[c*nb_per_cat:(c+1)*nb_per_cat, c*nb_per_cat:(c+1)*nb_per_cat]  # Shape: (50, 50)
    cat_RDM2 = RDM2[c*nb_per_cat:(c+1)*nb_per_cat, c*nb_per_cat:(c+1)*nb_per_cat]  # Shape: (50, 50)

    # Generate combinations of image indices
    all_combinations = list(combinations(range(nb_per_cat), images_per_subset))

    print(f"Testing {len(all_combinations)} combinations of {images_per_subset} images")

    best_indices = None
    best_model1_rdm = None
    best_model2_rdm = None
    best_similarity = np.inf

    # Test each combination
    for combination in tqdm(all_combinations, desc="Testing combinations", leave=False, position=1):
        indices = np.array(combination)
        # Get subset of activations
        rdm1 = cat_RDM1[np.ix_(indices, indices)]  # Shape: (4, 4)
        rdm2 = cat_RDM2[np.ix_(indices, indices)]  # Shape: (4, 4)

        # Compute similarity between RDMs
        similarity = rsa.Compute_sim_RDMs(rdm1, rdm2, center = False, metric = 'pearson_global', means= means)

        # Update best if this is better
        if similarity < best_similarity:
            best_indices = indices
            best_model1_rdm = rdm1
            best_model2_rdm = rdm2
            best_similarity = similarity

    print(f"Best indices for {category}: {best_indices}")
    print(f"Similarity: {best_similarity:.4f}")


In [ ]:
np.corrcoef(RDM1.flatten(), RDM2.flatten()).shape

In [ ]:
max_dissimilarity_images = max_rsa.find_max_dissimilarity_images(
                cat_activations, submodels, labels[:12], nb_per_cat,
                images_per_subset=4, similarity_metric='pearson', diff = maxdiffs
            )
similarity_dict = max_rsa.compute_sub_rdm_similarity(
            max_dissimilarity_images, cat_activations, submodels, labels[:12],
            savename = '')

In [ ]:
def sample_rdm_pairs(RDM1, RDM2, n_samples=100000, subset_size=40,
                                    batch_size=10000, seed=None):
    """
    Memory-efficient version that processes in batches and optionally saves to disk.

    Parameters:
    -----------
    batch_size : int
        Number of samples to process at once (default: 1000)
    output_file : str, optional
        If provided, saves results to this file using pickle
    """

    if seed is not None:
        np.random.seed(seed)

    n_images = RDM1.shape[0]
    n_batches = (n_samples + batch_size - 1) // batch_size

    all_sims_samples = []
    all_indices = []
    print(f"Processing {n_samples} samples in {n_batches} batches of {batch_size}...")

    for batch_idx in tqdm_notebook(range(n_batches)):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, n_samples)
        current_batch_size = end_idx - start_idx

        # Allocate batch arrays
        batch_sim = np.zeros((current_batch_size))
        batch_indices = np.zeros((current_batch_size, subset_size), dtype=int)

        for i in range(current_batch_size):
            # Randomly select images
            indices = np.random.choice(n_images, size=subset_size, replace=False)
            indices = np.sort(indices)

            # Extract submatrices
            batch_sim[i] = rsa.Compute_sim_RDMs(RDM1[np.ix_(indices, indices)], RDM2[np.ix_(indices, indices)], center = False, metric = 'pearson' )
            batch_indices[i] = indices

        all_sims_samples.append(batch_sim)
        all_indices.append(batch_indices)

    # Concatenate all batches
    sim_samples = np.concatenate(all_sims_samples, axis=0)
    indices_used = np.concatenate(all_indices, axis=0)


    return sim_samples, indices_used

In [ ]:
from itertools import combinations
def find_max_dissimilarity_images(cat_activations, models, categories, nb_per_cat,
                                  images_per_subset=4, dissimilarity_metric = 'L2squared', diff = np.array([0])):
    """
    Find the subset of images per category that maximizes RDM dissimilarity between two models.

    Parameters:
    -----------
    cat_activations : dict
        Dictionary with structure: cat_activations[model][category] = array of activations (n_images, n_features)
    models : list
        List of two model names, e.g., ['model1', 'model2']
    categories : list
        List of category names/indices
    compute_RDM : function
        Function that takes activations and returns RDM: RDM = compute_RDM(activations)
    images_per_subset : int
        Number of images to select per category (default: 4)
    method : str
        'exhaustive' or 'random' sampling of combinations

    Returns:
    --------
    results : dict
        Dictionary with results for each category:
        {
            category: {
                'best_indices': array of selected image indices,
                'max_dissimilarity': maximum dissimilarity value,
                'model1_rdm': RDM for model1 with selected images,
                'model2_rdm': RDM for model2 with selected images,
                'similarity': similarity between the two RDMs
            }
        }
    """

    if len(models) != 2:
        raise ValueError("This function requires exactly 2 models")

    results = {}

    #### First build the RDMs using all images of the chosen categories to get the general stats
    cat_activations_subset1 = cat_activations[models[0]][categories]
    cat_activations_subset2 = cat_activations[models[1]][categories]

    cat_shape = cat_activations_subset1.shape

    RDM1 = rsa.compute_RDMs(cat_activations_subset1.reshape(cat_shape[0]*cat_shape[1], -1), metric = dissimilarity_metric, display = False)
    RDM2 = rsa.compute_RDMs(cat_activations_subset2.reshape(cat_shape[0] * cat_shape[1], -1),
                            metric=dissimilarity_metric, display=False)
    means = {}
    n = len(RDM1)
    upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
    means['x'] = np.mean(RDM1[upper_indices])
    means['y'] = np.mean(RDM2[upper_indices])
    means['norm'] = np.std(RDM1[upper_indices]) * np.std(RDM2[upper_indices])
    print(means)
    for c, category in enumerate(tqdm_notebook(categories, desc="Processing categories")):
        print(f"\nProcessing category: {category}")
        # Get activations for both models for this category
        cat_RDM1 = RDM1[c*nb_per_cat:(c+1)*nb_per_cat, c*nb_per_cat:(c+1)*nb_per_cat]  # Shape: (50, 50)
        cat_RDM2 = RDM2[c*nb_per_cat:(c+1)*nb_per_cat, c*nb_per_cat:(c+1)*nb_per_cat]  # Shape: (50, 50)

        # Generate combinations of image indices
        all_combinations = list(combinations(range(nb_per_cat), images_per_subset))

        print(f"Testing {len(all_combinations)} combinations of {images_per_subset} images")

        best_indices = None
        best_model1_rdm = None
        best_model2_rdm = None
        best_similarity = 1

        # Test each combination
        for combination in tqdm_notebook(all_combinations, desc="Testing combinations", leave=False, position=1):
            indices = np.array(combination)
            # Get subset of activations
            rdm1 = cat_RDM1[np.ix_(indices, indices)]  # Shape: (4, 4)
            rdm2 = cat_RDM2[np.ix_(indices, indices)]  # Shape: (4, 4)

            n = len(rdm1)
            upper_indices = np.triu_indices(n, k=1)  # k=1 excludes diagonal
            if diff[c] <0:
                similarity = -np.mean(rdm1[upper_indices]) + np.mean(rdm2[upper_indices]) +  2*np.std(rdm1[upper_indices]) + np.std(rdm2[upper_indices])
            else:
                similarity = np.mean(rdm1[upper_indices]) - np.mean(rdm2[upper_indices]) +  2*np.std(rdm1[upper_indices]) + np.std(rdm2[upper_indices])

            # Update best if this is better
            if similarity < best_similarity:
                best_indices = indices
                best_model1_rdm = rdm1
                best_model2_rdm = rdm2
                best_similarity = similarity

        # Store results for this category
        results[category] = {
            'best_indices': best_indices,
            'model1_rdm': best_model1_rdm,
            'model2_rdm': best_model2_rdm,
            'similarity': best_similarity
        }

        print(f"Best indices for {category}: {best_indices}")
        print(f"Similarity: {best_similarity:.4f}")

    return results

In [ ]:
labels, sortedmaxdiffcats, maxdiffs = max_rsa.max_compactness_difference(
                compact_categories, compactness, 565, listcat, models = ['saycam', 'supervised'],
                nb_considered_categories = 12, compactness_diff_measure = 'normalizedDiff'
            )
max_dissimilarity_images = find_max_dissimilarity_images(
                cat_activations, ['saycam', 'supervised'], labels[:12], 50,
                images_per_subset=4, diff = maxdiffs
            )
similarity_dict = max_rsa.compute_sub_rdm_similarity(
            max_dissimilarity_images, cat_activations, ['saycam', 'supervised'], labels[:12],
            savename = '')